# SATELLITE DATA ANALYSIS FOR OUR STARTUP

In [1]:
# Instalamos todo lo necesario
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
import json
import requests
import time
import os
!pip install pandas openpyxl
!pip install python-dotenv

In [2]:
# Importamos el dataset
df_ucs = pd.read_excel(r'C:\Users\Usuario\Desktop\Ironhack\Bootcamp\03_Week3\satellite_data_analysis\Datasets\UCS_Satellite_Database.xlsx')

In [3]:
df_ucs.head()

,Current Official Name of Satellite,Country/Org of UN Registry,Country of Operator/Owner,Operator/Owner,Users,Purpose,Detailed Purpose,Class of Orbit,Type of Orbit,Longitude of GEO (degrees),...,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66
0,1HOPSAT-TD,NR,USA,Hera Systems,Commercial,Earth Observation,Infrared Imaging,LEO,Non-Polar Inclined,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAC AIS-Sat1 (Kelpie 1),United Kingdom,United Kingdom,AAC Clyde Space,Commercial,Earth Observation,Automatic Identification System (AIS),LEO,Sun-Synchronous,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Aalto-1,Finland,Finland,Aalto University,Civil,Technology Development,NaN,LEO,Sun-Synchronous,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAt-4,Denmark,Denmark,University of Aalborg,Civil,Earth Observation,Automatic Identification System (AIS),LEO,Sun-Synchronous,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ABS-2,NR,Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,75.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_ucs.shape

(7560, 67)

In [5]:
df_ucs.columns

Index(['Current Official Name of Satellite', 'Country/Org of UN Registry',
       'Country of Operator/Owner', 'Operator/Owner', 'Users', 'Purpose',
       'Detailed Purpose', 'Class of Orbit', 'Type of Orbit',
       'Longitude of GEO (degrees)', 'Perigee (km)', 'Apogee (km)',
       'Eccentricity', 'Inclination (degrees)', 'Period (minutes)',
       'Launch Mass (kg.)', 'Dry Mass (kg.)', 'Power (watts)',
       'Date of Launch', 'Expected Lifetime (yrs.)', 'Contractor',
       'Country of Contractor', 'Launch Site', 'Launch Vehicle',
       'COSPAR Number', 'NORAD Number', 'Comments', 'Unnamed: 27',
       'Source Used for Orbital Data', 'Source', 'Source.1', 'Source.2',
       'Source.3', 'Source.4', 'Source.5', 'Source.6', 'Unnamed: 36',
       'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40',
       'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44',
       'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47', 'Unnamed: 48',
       'Unnamed: 49', 'Unnamed: 50', 'Unnamed: 

In [6]:
df_ucs.loc[7186, 'Date of Launch'] = '01/09/2023' # Cambiamos el formato de una fehca
df_ucs = df_ucs.drop([240, 349]) # Eliminamos dos filas donde no sale la fecha

In [7]:
df_ucs['Date of Launch'] = pd.to_datetime(df_ucs['Date of Launch']) # Convertimos todas las fechas a datetime

In [8]:
df_ucs.head()

,Current Official Name of Satellite,Country/Org of UN Registry,Country of Operator/Owner,Operator/Owner,Users,Purpose,Detailed Purpose,Class of Orbit,Type of Orbit,Longitude of GEO (degrees),...,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66
0,1HOPSAT-TD,NR,USA,Hera Systems,Commercial,Earth Observation,Infrared Imaging,LEO,Non-Polar Inclined,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAC AIS-Sat1 (Kelpie 1),United Kingdom,United Kingdom,AAC Clyde Space,Commercial,Earth Observation,Automatic Identification System (AIS),LEO,Sun-Synchronous,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Aalto-1,Finland,Finland,Aalto University,Civil,Technology Development,NaN,LEO,Sun-Synchronous,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAt-4,Denmark,Denmark,University of Aalborg,Civil,Earth Observation,Automatic Identification System (AIS),LEO,Sun-Synchronous,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ABS-2,NR,Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,75.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df_ucs["Launch Site"].unique()

array(['Satish Dhawan Space Centre', 'Cape Canaveral',
       'Guiana Space Center', 'Baikonur Cosmodrome', 'Vandenberg AFB',
       'Rocket Lab Launch Complex 1', 'Cygnus', 'Dombarovsky Air Base',
       'Xichang Satellite Launch Center', 'Vostochny Cosmodrome',
       'Uchinoura Space Center', 'Jiuquan Satellite Launch Center',
       'Plesetsk Cosmodrome', 'Taiyuan Launch Center', 'Virgin Orbit',
       'Vandeberg AFB', 'Yellow Sea Launch Platform',
       'FANTM-RAiL [Xtenti]',
       'Mid-Atlantic Regional Spaceport/Wallops Island',
       'Wenchang Space Center', 'Orbital ATK L-1011',
       'Tanegashima Space Center', 'International Space Station',
       'Antares', 'Sea Launch Odyssey', 'Svobodny Cosmodrome',
       'Kodiak Launch Complex', 'Wallops Island Flight Facility',
       'Stargazer L-1011', 'Kwajalein Island', 'Taiyan Launch Center',
       'International Space Station - Cygnus', 'Shahroud Missile Range',
       'Palmachim Launch Complex ', 'Satish Dhawan Space Center

In [10]:
df_ucs["Launch Site"].nunique()

40

In [11]:
df_ucs["Launch Site"] = df_ucs["Launch Site"].str.strip()

In [12]:
# Unificar las distintas formas de escribir 'Rocket Lab Launch Complex 1B'
df_ucs["Launch Site"] = df_ucs["Launch Site"].replace("Rocket Lab Launch Complex 1B", "Rocket Lab Launch Complex 1")

In [13]:
# Unificar 'FANTM-RAiL (Xtenti)' y 'FANTM-RAiL [Xtenti]' y pasarlo a 'Cape Canaveral' ya que he investigado, y esta empresa usa esa localización
df_ucs["Launch Site"] = df_ucs["Launch Site"].replace(['FANTM-RAiL (Xtenti)', 'FANTM-RAiL [Xtenti]'], 'Cape Canaveral')

In [14]:
# Reemplazar múltiples localizaciones por 'Plain/ISS Launch' ya que no puedo obtener el clima en esos sitios ya que no son terrestres
plane_iss_launch = ['Orbital ATK L-1011','International Space Station - Cygnus','International Space Station','Cygnus','Stargazer L-1011','Virgin Orbit']
df_ucs["Launch Site"] = df_ucs["Launch Site"].replace(plane_iss_launch, "Plane/ISS Launch")

In [15]:
# Unificar Vandenber AFB
df_ucs['Launch Site'] = df_ucs['Launch Site'].replace('Vandeberg AFB', 'Vandenberg AFB')

In [16]:
# Unificar Satish Dhawan Space Center
df_ucs['Launch Site'] = df_ucs['Launch Site'].replace('Satish Dhawan Space Centre', 'Satish Dhawan Space Center')

In [17]:
# Unificar Kodiak Launch Complex
df_ucs['Launch Site'] = df_ucs['Launch Site'].replace('Kodiak Island', 'Kodiak Launch Complex')

In [18]:
# Unificar Wenchang Space Launch Site
df_ucs["Launch Site"] = df_ucs["Launch Site"].replace(['Wenchang Satellite Launch Center', 'Wenchang Space Center'], 'Wenchang Space Launch Site')

In [19]:
# Corregir valores
df_ucs['Launch Site'] = df_ucs['Launch Site'].replace('Taiyan Launch Center', 'Taiyuan Launch Center')

In [20]:
df_ucs["Launch Site"].unique()

array(['Satish Dhawan Space Center', 'Cape Canaveral',
       'Guiana Space Center', 'Baikonur Cosmodrome', 'Vandenberg AFB',
       'Rocket Lab Launch Complex 1', 'Plane/ISS Launch',
       'Dombarovsky Air Base', 'Xichang Satellite Launch Center',
       'Vostochny Cosmodrome', 'Uchinoura Space Center',
       'Jiuquan Satellite Launch Center', 'Plesetsk Cosmodrome',
       'Taiyuan Launch Center', 'Yellow Sea Launch Platform',
       'Mid-Atlantic Regional Spaceport/Wallops Island',
       'Wenchang Space Launch Site', 'Tanegashima Space Center',
       'Antares', 'Sea Launch Odyssey', 'Svobodny Cosmodrome',
       'Kodiak Launch Complex', 'Wallops Island Flight Facility',
       'Kwajalein Island', 'Shahroud Missile Range',
       'Palmachim Launch Complex', 'Naro Space Center'], dtype=object)

In [21]:
# Diccionario de las coordenadas de cada valor único de la columna 'Launch Site' con los valores únicos
launch_site_coordinate = {
    'Vandenberg AFB': (34.74738558053763, -120.57287382598979),
    'Cape Canaveral': (28.478816053592052, -80.57606021672525),
    'Guiana Space Center': (5.23314401858844, -52.776983796694616),
    'Baikonur Cosmodrome': (45.96725377748388, 63.30547994996387),
    'Wallops Island Flight Facility': (37.933875469177956, -75.47227890015118),
    'Satish Dhawan Space Center': (13.726115425221352, 80.22658668288408),
    'Sea Launch Odyssey': (33.75803115528897, -118.21714540854131),
    'Svobodny Cosmodrome': (51.79375147822065, 128.18611741169877),
    'Plesetsk Cosmodrome': (62.92795253294236, 40.574883713321356),
    'Palmachim Launch Complex': (31.89933331285537, 34.69069702671702),
    'Taiyuan Launch Center': (38.848744585386704, 111.60792935276686),
    'Xichang Satellite Launch Center': (28.246289127653373, 102.02655974251424),
    'Jiuquan Satellite Launch Center': (40.95820141282573, 100.29113255468557),
    'Dombarovsky Air Base': (50.800636829015794, 59.51676155312275),
    'Uchinoura Space Center': (31.251286366566344, 131.07612469723395),
    'Kwajalein Island': (8.720654879025744, 167.73129220584238),
    'Tanegashima Space Center': (30.37513340280436, 130.95763536784997),
    'Kodiak Launch Complex': (57.43532950799345, -152.33958590527274),
    'Naro Space Center': (34.45356111005476, 127.51782879680057),
    'Vostochny Cosmodrome': (51.84953794993724, 128.35530328395734),
    'Wenchang Space Launch Site': (19.619733115878066, 110.95159716666655),
    'Rocket Lab Launch Complex 1': (-39.26076827278987, 177.8658598361479),
    'Yellow Sea Launch Platform': (35.01725874873759, 122.99900258520917),
    'Shahroud Missile Range': (36.20102786793409, 55.33395241490814),
    'Mid-Atlantic Regional Spaceport/Wallops Island': (37.93221269267054, -75.4813545300791),
    'Antares': (37.932076857305944, -75.48138113038931),
    'Plane/ISS Launch': None,
}

# Asignamos las coordenadas en la nueva columna 'Geographic Coordinate'
df_ucs['Geographic Coordinate'] = df_ucs['Launch Site'].map(launch_site_coordinate)

In [23]:
df_ucs.columns

Index(['Current Official Name of Satellite', 'Country/Org of UN Registry',
       'Country of Operator/Owner', 'Operator/Owner', 'Users', 'Purpose',
       'Detailed Purpose', 'Class of Orbit', 'Type of Orbit',
       'Longitude of GEO (degrees)', 'Perigee (km)', 'Apogee (km)',
       'Eccentricity', 'Inclination (degrees)', 'Period (minutes)',
       'Launch Mass (kg.)', 'Dry Mass (kg.)', 'Power (watts)',
       'Date of Launch', 'Expected Lifetime (yrs.)', 'Contractor',
       'Country of Contractor', 'Launch Site', 'Launch Vehicle',
       'COSPAR Number', 'NORAD Number', 'Comments', 'Unnamed: 27',
       'Source Used for Orbital Data', 'Source', 'Source.1', 'Source.2',
       'Source.3', 'Source.4', 'Source.5', 'Source.6', 'Unnamed: 36',
       'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40',
       'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44',
       'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47', 'Unnamed: 48',
       'Unnamed: 49', 'Unnamed: 50', 'Unnamed: 

In [24]:
# Eliminar columnas que no necesito
df_ucs.drop(['Source',
       'Source.1', 'Source.2', 'Source.3', 'Source.4', 'Source.5', 'Source.6',
       'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39',
       'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43',
       'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47',
       'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50', 'Unnamed: 51',
       'Unnamed: 52', 'Unnamed: 53', 'Unnamed: 54', 'Unnamed: 55',
       'Unnamed: 56', 'Unnamed: 57', 'Unnamed: 58', 'Unnamed: 59',
       'Unnamed: 60', 'Unnamed: 61', 'Unnamed: 62', 'Unnamed: 63',
       'Unnamed: 64', 'Unnamed: 65', 'Unnamed: 66'], axis=1, inplace=True)


In [25]:
df_ucs.columns

Index(['Current Official Name of Satellite', 'Country/Org of UN Registry',
       'Country of Operator/Owner', 'Operator/Owner', 'Users', 'Purpose',
       'Detailed Purpose', 'Class of Orbit', 'Type of Orbit',
       'Longitude of GEO (degrees)', 'Perigee (km)', 'Apogee (km)',
       'Eccentricity', 'Inclination (degrees)', 'Period (minutes)',
       'Launch Mass (kg.)', 'Dry Mass (kg.)', 'Power (watts)',
       'Date of Launch', 'Expected Lifetime (yrs.)', 'Contractor',
       'Country of Contractor', 'Launch Site', 'Launch Vehicle',
       'COSPAR Number', 'NORAD Number', 'Comments', 'Unnamed: 27',
       'Source Used for Orbital Data', 'Geographic Coordinate'],
      dtype='object')

In [26]:
# Recordenar columnas
nuevo_orden = ['Date of Launch', 'Geographic Coordinate', 'COSPAR Number', 'NORAD Number','Current Official Name of Satellite', 'Launch Site', 'Country/Org of UN Registry',
       'Country of Operator/Owner', 'Operator/Owner', 'Launch Vehicle', 'Expected Lifetime (yrs.)',
       'Contractor', 'Country of Contractor', 'Users', 'Purpose',
       'Detailed Purpose', 'Class of Orbit', 'Type of Orbit',
       'Longitude of GEO (degrees)', 'Perigee (km)', 'Apogee (km)',
       'Eccentricity', 'Inclination (degrees)', 'Period (minutes)',
       'Launch Mass (kg.)', 'Dry Mass (kg.)', 'Power (watts)', 
       'Comments', 'Unnamed: 27', 'Source Used for Orbital Data',
       ] 
df_ucs = df_ucs[nuevo_orden]


In [27]:
# Ordenamos el df en orden de más antiguo o más reciente
df_ucs = df_ucs.sort_values(by='Date of Launch')

In [28]:
# Para restablecer el índice
df_ucs = df_ucs.reset_index(drop=True)

In [29]:
df_ucs.head()

,Date of Launch,Geographic Coordinate,COSPAR Number,NORAD Number,Current Official Name of Satellite,Launch Site,Country/Org of UN Registry,Country of Operator/Owner,Operator/Owner,Launch Vehicle,...,Apogee (km),Eccentricity,Inclination (degrees),Period (minutes),Launch Mass (kg.),Dry Mass (kg.),Power (watts),Comments,Unnamed: 27,Source Used for Orbital Data
0,1974-11-15,"(34.74738558053763, -120.57287382598979)",1974-089B,7530,Amsat-Oscar 7,Vandenberg AFB,USA,USA,AMSAT-NA,Delta 2310,...,1459.0,0.001215,101.40,114.90,29.0,NaN,NaN,American amateur radio satellite.,NaN,www.satellitedebris.net 12/12
1,1988-09-29,"(28.478816053592052, -80.57606021672525)",1988-091B,19548,TDRS-3,Cape Canaveral,USA,USA,National Aeronautics and Space Administration ...,Space Shuttle (STS 26),...,35878.0,0.002194,11.53,1436.06,3180.0,1600,1800,Backup; still partially operational.,NaN,JM/12_08
2,1989-09-25,"(28.478816053592052, -80.57606021672525)",1989-077A,20253,USA 46,Cape Canaveral,USA,USA,US Navy,Atlas Centaur,...,35829.0,0.000996,8.97,1436.13,2310.0,1884,NaN,Old system replaced by UFO satellites; this sa...,NaN,JM/2_09
3,1990-04-25,"(28.478816053592052, -80.57606021672525)",1990-037B,20580,Hubble Space Telescope,Cape Canaveral,USA,ESA,European Space Agency (ESA)/NASA,Space Shuttle (STS 31),...,559.0,0.000289,28.50,95.80,11110.0,NaN,2400,Exploration of space.,NaN,www.satellitedebris.net 12/12
4,1990-08-30,"(5.23314401858844, -52.776983796694616)",1990-079A,20776,Skynet 4C,Guiana Space Center,United Kingdom,United Kingdom,Intelsat/Paradigm Secure Communications (wholl...,Ariane 44LP,...,35787.0,0.000024,13.60,1436.07,1474.0,850,1200,Spare. In March 2010 it was announced that the...,NaN,JM/12_08


# API OPEN-METEO.COM PARA INCLUIR EL CLIMA (CON DISTINTAS VARIABLES) DEL LUGAR EXACTO SEGÚN LAS COORDENADAS Y SEGÚN EL DÍA

Voy a extraer de la API open-meteo.com todas estas variables según las coordenadas exactas y la fecha exacta (son daily weather variables): 

Weather code, Mean Temperature (2 m), Precipitation Sum, Precipitation Hours, Maximum Wind Speed (10 m), Maximum Wind Gusts (10 m), Mean Cloud cover, Mean Precipitation Probability, Mean Relative Humidity (2 m), Mean Visibility

In [30]:
# Muestra tipos únicos en la columna
print(df_ucs['Geographic Coordinate'].apply(type).value_counts())


Geographic Coordinate
<class 'tuple'>       7503
<class 'NoneType'>      55
Name: count, dtype: int64


In [37]:
# Filtra para obtener solo las que no son nulos (notna para que devuelva true o false)
df_ucs_valid_coordinate = df_ucs[df_ucs['Geographic Coordinate'].notna()]

# Extraemos latitud y longitud de la columna 'Geographic Coordinate' (que la convertimos a string con astype(str), ya que no es un string, es una tupla (con 2 floats dentro) y así podremos usar strip/split)
# Eliminamos los paréntesis. Dividimos la cadena en 2 con split. Convetimos a float.
df_ucs_valid_coordinate[['lat', 'lon']] = df_ucs_valid_coordinate['Geographic Coordinate'].astype(str).str.strip('()').str.split(', ', expand=True).astype(float)

# Formateamos la fecha a string para usarla en el request (dt es para acceder a distintas cosas dentro de datetime)
df_ucs_valid_coordinate['date'] = df_ucs_valid_coordinate['Date of Launch'].dt.strftime('%Y-%m-%d')


C:\Users\Usuario\AppData\Local\Temp\ipykernel_13680\3739916105.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ucs_valid_coordinate[['lat', 'lon']] = df_ucs_valid_coordinate['Geographic Coordinate'].astype(str).str.strip('()').str.split(', ', expand=True).astype(float)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_13680\3739916105.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ucs_valid_coordinate[['lat', 'lon']] = df_ucs_valid_coordinate['Geographic Coordinate'].astype(str).str.strip('()').s

In [38]:
df_ucs_valid_coordinate.columns

Index(['Date of Launch', 'Geographic Coordinate', 'COSPAR Number',
       'NORAD Number', 'Current Official Name of Satellite', 'Launch Site',
       'Country/Org of UN Registry', 'Country of Operator/Owner',
       'Operator/Owner', 'Launch Vehicle', 'Expected Lifetime (yrs.)',
       'Contractor', 'Country of Contractor', 'Users', 'Purpose',
       'Detailed Purpose', 'Class of Orbit', 'Type of Orbit',
       'Longitude of GEO (degrees)', 'Perigee (km)', 'Apogee (km)',
       'Eccentricity', 'Inclination (degrees)', 'Period (minutes)',
       'Launch Mass (kg.)', 'Dry Mass (kg.)', 'Power (watts)', 'Comments',
       'Unnamed: 27', 'Source Used for Orbital Data', 'lat', 'lon', 'date'],
      dtype='object')

In [41]:
# Definimosla al función para consultar la API de open-meteo
def open_meteo_data(lat, lon, date):
    url = (
        "https://archive-api.open-meteo.com/v1/archive?"
        f"latitude={lat}&longitude={lon}&start_date={date}&end_date={date}"
        "&daily=weathercode,temperature_2m_mean,precipitation_sum,"
        "precipitation_hours,windspeed_10m_max,windgusts_10m_max,"
        "cloudcover_mean,precipitation_probability_mean,"
        "relative_humidity_2m_mean,visibility_mean"
        "&timezone=UTC"
    ) # Url de la API de open-meteo con los parámetros que necesito

    try:
        res = requests.get(url) # La petición a la API
        res.raise_for_status()  # Dame error si la respuesta no es 200
        data = res.json() # Tenemos que pasarlo a json para poder hacer un diccionario en python

        if 'daily' not in data or not data['daily']['time']:
            return [None] * 10  # Para ver si hay datos en daily, si no hay datos, que nos de None

        d = data['daily'] # Para guardar en la variable d el contenido de 'daily' del JSON que devuelve la API

        return [
            d['weathercode'][0],
            d['temperature_2m_mean'][0],
            d['precipitation_sum'][0],
            d['precipitation_hours'][0],
            d['windspeed_10m_max'][0],
            d['windgusts_10m_max'][0],
            d['cloudcover_mean'][0],
            d['precipitation_probability_mean'][0],
            d['relative_humidity_2m_mean'][0],
            d['visibility_mean'][0]
        ] # Para obtener los valores de cada variable

    except Exception as e:
        print(f"Error en {lat}, {lon}, {date}: {e}")
        return [None] * 10 # Nos da None para cada uno de los 10 valores

In [ ]:
# Columnas que quiero añadir
columns = [
    'weathercode', 'temperature_2m_mean', 'precipitation_sum', 'precipitation_hours',
    'windspeed_10m_max', 'windgusts_10m_max', 'cloudcover_mean', 'precipitation_probability_mean',
    'relative_humidity_2m_mean', 'visibility_mean'
]

# Creamos las columnas vacías en el df
df_ucs_valid_coordinate[columns] = None

C:\Users\Usuario\AppData\Local\Temp\ipykernel_13680\1257337070.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ucs_valid_coordinate[columns] = None
C:\Users\Usuario\AppData\Local\Temp\ipykernel_13680\1257337070.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ucs_valid_coordinate[columns] = None
C:\Users\Usuario\AppData\Local\Temp\ipykernel_13680\1257337070.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [42]:
# Iterar sobre cada fila en el nuevo df: df_ucs_valid_coordinate
for i, row in df_ucs_valid_coordinate.iterrows():
    # Obtener los datos meteorológicos desde Open-Meteo
    weather_data = open_meteo_data(row['lat'], row['lon'], row['date'])
    
    # Asignar los datos obtenidos a las columnas correspondientes
    df_ucs_valid_coordinate.loc[i, columns] = weather_data

    # Pausa entre las peticiones para no sobrecargar el servidor
    time.sleep(0.5)

# API de n2yo.com

In [44]:
# Para guardar la API Key
from dotenv import load_dotenv, find_dotenv

load_dotenv('n2yo_key.env')

n2yo_key = os.getenv("n2yo_key")

In [45]:
# Para ver Two-Line Element Set (TLE), un formato estándar que proporciona los elementos orbitales de un satélite para predecir su posición en el espacio

satellite_id = 25544
api_key = n2yo_key 

url = f"https://api.n2yo.com/rest/v1/satellite/tle/{satellite_id}?apiKey={api_key}"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    tle_lines = data.get('tle', '').split("\r\n")
    satellite_name = data['info'].get("satname", "Unknown")
    
    if len(tle_lines) == 2:
        df_tle = pd.DataFrame([[satellite_id] + tle_lines + [satellite_name]], columns=["Satellite ID", "TLE Line 1", "TLE Line 2",  "Satellite Name"]) 
        display(df_tle)

,Satellite ID,TLE Line 1,TLE Line 2,Satellite Name
0,25544,1 25544U 98067A 25126.47897065 .00009245 0...,2 25544 51.6347 153.6603 0002330 83.4740 276...,SPACE STATION


In [46]:
# A través del TLE anterior, y con Skyfield, podemos calcular la posición del satélite
!pip install skyfield
!pip install skyfield reverse_geocoder pandas
!pip install pytz # Para convertir la hora UCT a la hora de España

ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


In [49]:
from skyfield.api import EarthSatellite, load
from datetime import datetime, timezone
import pandas as pd
import reverse_geocoder as rg
import pytz

# Con las columnas del df_tle: "Satellite ID", "TLE Line 1", "TLE Line 2"
tle_line1 = df_tle.loc[0, "TLE Line 1"]
tle_line2 = df_tle.loc[0, "TLE Line 2"]
satellite_id = df_tle.loc[0, "Satellite ID"]
# Crear el objeto del satélite
ts = load.timescale() 
satellite = EarthSatellite(tle_line1, tle_line2, f'Satellite {satellite_id}', ts)
# Tiempo actual con zona horaria UTC
now_utc = datetime.now(timezone.utc)
ts = load.timescale()
t = ts.utc(now_utc)
# Obtener posición
geocentric = satellite.at(t)
subpoint = geocentric.subpoint()
# Datos de latitud, longitud y altitud
lat, lon, elev = subpoint.latitude.degrees, subpoint.longitude.degrees, subpoint.elevation.km
# Buscar ciudad y país usando reverse geocoding
location = rg.search((lat, lon))[0]
city, country = location['name'], location['cc']
# Imprimir resultados
print(f"Hora UTC: {now_utc.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Posición actual del satélite:\nLatitud: {lat:.2f}°\nLongitud: {lon:.2f}°\nAltitud: {elev:.2f} km")
print(f"Está sobre: {city}, {country}")

Hora UTC: 2025-05-06 18:34:04
Posición actual del satélite:
Latitud: -19.51°
Longitud: -154.99°
Altitud: 424.58 km
Está sobre: Vaitape, PF
